In [60]:
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from typing import List, Annotated, TypedDict
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langchain_core.language_models.chat_models import BaseChatModel
from typing import Literal
import warnings
warnings.filterwarnings("ignore")
print("Running code with warnings suppressed.")
        
class State(MessagesState):
    next: str

# Tool for searching diet and exercise information
diet_exercise_tool = TavilySearchResults(max_results=5)

@tool
def log_meal(meal_description: str) -> str:
    "Log the meal described by the user."
    # Here we would implement the logic to log the meal
    return "Meal logged: " + meal_description


### Step 2: Create Agent Nodes

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(model="gpt-4o")

# Diet Agent
diet_agent = create_react_agent(llm, tools=[diet_exercise_tool])

def diet_node(state: State) -> Command[Literal["supervisor"]]:
    result = diet_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="diet_agent")
            ]
        },
        goto="supervisor",
    )

# Exercise Agent
exercise_agent = create_react_agent(llm, tools=[diet_exercise_tool])

def exercise_node(state: State) -> Command[Literal["supervisor"]]:
    result = exercise_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="exercise_agent")
            ]
        },
        goto="supervisor",
    )

# Logging Agent
logging_agent = create_react_agent(llm, tools=[log_meal])

def logging_node(state: State) -> Command[Literal["supervisor"]]:
    result = logging_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="logging_agent")
            ]
        },
        goto="supervisor",
    )

### Step 3: Create Supervisor Node

def make_supervisor_node(llm: BaseChatModel, members: list[str]) -> str:
    options = ["FINISH"] + members
    system_prompt = (
        "You are a supervisor tasked with managing a conversation between the"
        f" following workers. Given the following user request,"
        " respond with the worker to act next. Each worker will perform a"
        " task and respond with their results and status. When finished,"
        " respond with FINISH."
    )

    class Router(TypedDict):
        "Worker to route to next. If no workers needed, route to FINISH."
        next: Literal["diet_agent", "exercise_agent", "logging_agent"]
        
    def supervisor_node(state: State) -> Command[Literal[*members, "__end__"]]:
        "An LLM-based router."
        messages = [
            {"role": "system", "content": system_prompt},
        ] + state["messages"]
        response = llm.with_structured_output(Router).invoke(messages)
        goto = response["next"]
        if goto == "FINISH":
            goto = END

        return Command(goto=goto, update={"next": goto})

    return supervisor_node


teams_supervisor_node = make_supervisor_node(llm, ["diet_agent", "exercise_agent", "logging_agent"])

### Step 4: Build the StateGraph

# Now we will build the `StateGraph` for the `Fitness Advisor` team.

# Create the graph
fitness_builder = StateGraph(State)
fitness_builder.add_node("supervisor", teams_supervisor_node)
fitness_builder.add_node("diet_agent", diet_node)
fitness_builder.add_node("exercise_agent", exercise_node)
fitness_builder.add_node("logging_agent", logging_node)

fitness_builder.add_edge(START, "supervisor")
fitness_graph = fitness_builder.compile()
print(fitness_graph.get_graph(xray=True).to_json())

Running code with warnings suppressed.
{'nodes': [{'id': '__start__', 'type': 'schema', 'data': '__start__'}, {'id': 'supervisor', 'type': 'runnable', 'data': {'id': ['langgraph', 'utils', 'runnable', 'RunnableCallable'], 'name': 'supervisor'}}, {'id': 'diet_agent:__start__', 'type': 'schema', 'data': 'diet_agent:__start__'}, {'id': 'diet_agent:agent', 'type': 'runnable', 'data': {'id': ['langgraph', 'utils', 'runnable', 'RunnableCallable'], 'name': 'diet_agent:agent'}}, {'id': 'diet_agent:tools', 'type': 'runnable', 'data': {'id': ['langgraph', 'prebuilt', 'tool_node', 'ToolNode'], 'name': 'diet_agent:tools'}}, {'id': 'diet_agent:__end__', 'type': 'schema', 'data': 'diet_agent:__end__'}, {'id': 'exercise_agent:__start__', 'type': 'schema', 'data': 'exercise_agent:__start__'}, {'id': 'exercise_agent:agent', 'type': 'runnable', 'data': {'id': ['langgraph', 'utils', 'runnable', 'RunnableCallable'], 'name': 'exercise_agent:agent'}}, {'id': 'exercise_agent:tools', 'type': 'runnable', 'data

In [11]:
python_code_snippet = """
import os
import json
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.tools import tool
from typing import List, Annotated, TypedDict
from langgraph.graph import StateGraph, MessagesState, START, END
from langgraph.types import Command
from langchain_core.language_models.chat_models import BaseChatModel
from typing import Literal
import warnings
warnings.filterwarnings("ignore")
print("Running code with warnings suppressed.")

os.environ["TAVILY_API_KEY"] = "your_tavily_api_key_here"
os.environ["OPENAI_API_KEY"] = "your_openai_api_key_here"


class State(MessagesState):
    next: str

# Tool for searching diet and exercise information
diet_exercise_tool = TavilySearchResults(max_results=5)

@tool
def log_meal(meal_description: str) -> str:
    "Log the meal described by the user."
    # Here we would implement the logic to log the meal
    return "Meal logged: " + meal_description


### Step 2: Create Agent Nodes

from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.prebuilt import create_react_agent

llm = ChatOpenAI(model="gpt-4o")

# Diet Agent
diet_agent = create_react_agent(llm, tools=[diet_exercise_tool])

def diet_node(state: State) -> Command[Literal["supervisor"]]:
    result = diet_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="diet_agent")
            ]
        },
        goto="supervisor",
    )

# Exercise Agent
exercise_agent = create_react_agent(llm, tools=[diet_exercise_tool])

def exercise_node(state: State) -> Command[Literal["supervisor"]]:
    result = exercise_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="exercise_agent")
            ]
        },
        goto="supervisor",
    )

# Logging Agent
logging_agent = create_react_agent(llm, tools=[log_meal])

def logging_node(state: State) -> Command[Literal["supervisor"]]:
    result = logging_agent.invoke(state)
    return Command(
        update={
            "messages": [
                HumanMessage(content=result["messages"][-1].content, name="logging_agent")
            ]
        },
        goto="supervisor",
    )

### Step 3: Create Supervisor Node

def make_supervisor_node(llm: BaseChatModel, members: list[str]) -> str:
    options = ["FINISH"] + members
    system_prompt = (
        "You are a supervisor tasked with managing a conversation between the"
        f" following workers. Given the following user request,"
        " respond with the worker to act next. Each worker will perform a"
        " task and respond with their results and status. When finished,"
        " respond with FINISH."
    )

    class Router(TypedDict):
        "Worker to route to next. If no workers needed, route to FINISH."
        next: Literal["FINISH", "diet_agent", "exercise_agent", "logging_agent"]
        
    def supervisor_node(state: State) -> Command[Literal["diet_agent", "exercise_agent", "logging_agent", "__end__"]]:
        "An LLM-based router."
        messages = [
            {"role": "system", "content": system_prompt},
        ] + state.messages
        response = llm.with_structured_output(Router).invoke(messages)
        goto = response["next"]
        if goto == "FINISH":
            goto = END

        return Command(goto=goto, update={"next": goto})

    return supervisor_node


teams_supervisor_node = make_supervisor_node(llm, ["diet_agent", "exercise_agent", "logging_agent"])

### Step 4: Build the StateGraph

# Now we will build the `StateGraph` for the `Fitness Advisor` team.

# Create the graph
fitness_builder = StateGraph(State)
fitness_builder.add_node("supervisor", teams_supervisor_node)
fitness_builder.add_node("diet_agent", diet_node)
fitness_builder.add_node("exercise_agent", exercise_node)
fitness_builder.add_node("logging_agent", logging_node)

fitness_builder.add_edge(START, "supervisor")
fitness_graph = fitness_builder.compile()

print(fitness_graph.get_graph(xray=True).to_json())

with open("/home/user/sample.json" , "w" ) as write:
    json.dump(fitness_graph.get_graph(xray=True).to_json(), write)
"""

In [22]:
from openevals.code.e2b.execution import create_e2b_execution_evaluator
from dotenv import load_dotenv
from e2b_code_interpreter import Sandbox

load_dotenv()
sandbox = Sandbox()
evaluator = create_e2b_execution_evaluator(
    sandbox=sandbox,
)

In [23]:
eval_result = evaluator(outputs=python_code_snippet)

In [24]:
eval_result

{'key': 'execution_succeeded',
 'score': True,
 'comment': None,
 'metadata': None}

In [25]:
files = sandbox.files.read("/home/user/sample.json")

In [26]:
print(files)

{"nodes": [{"id": "__start__", "type": "schema", "data": "__start__"}, {"id": "supervisor", "type": "runnable", "data": {"id": ["langgraph", "utils", "runnable", "RunnableCallable"], "name": "supervisor"}}, {"id": "diet_agent:__start__", "type": "schema", "data": "diet_agent:__start__"}, {"id": "diet_agent:agent", "type": "runnable", "data": {"id": ["langgraph", "utils", "runnable", "RunnableCallable"], "name": "diet_agent:agent"}}, {"id": "diet_agent:tools", "type": "runnable", "data": {"id": ["langgraph", "prebuilt", "tool_node", "ToolNode"], "name": "diet_agent:tools"}}, {"id": "diet_agent:__end__", "type": "schema", "data": "diet_agent:__end__"}, {"id": "exercise_agent:__start__", "type": "schema", "data": "exercise_agent:__start__"}, {"id": "exercise_agent:agent", "type": "runnable", "data": {"id": ["langgraph", "utils", "runnable", "RunnableCallable"], "name": "exercise_agent:agent"}}, {"id": "exercise_agent:tools", "type": "runnable", "data": {"id": ["langgraph", "prebuilt", "to

In [21]:
with open('sample.json', 'w') as file:
    file.write(files)

In [17]:
print(files)

# Old method which does not allow auto import

In [117]:
execution = sandbox.run_code(python_code_snippet) # Execute Python inside the sandbox
print(execution.logs)

Logs(stdout: [], stderr: [])


In [106]:
import json 
json_result = json.loads(eval_result["comment"])

for error in json_result:
    pprint(error)

{'message': '"__getitem__" method not defined on type "BaseModel"',
 'range': {'end': {'character': 23, 'line': 102},
           'start': {'character': 15, 'line': 102}},
 'rule': 'reportIndexIssue',
 'severity': 'error'}
{'message': 'Type "(state: State) -> Command[Literal[\'diet_agent\', '
            '\'exercise_agent\', \'logging_agent\', \'__end__\']]" is not '
            'assignable to return type "str"\n'
            '\xa0\xa0"function" is not assignable to "str"',
 'range': {'end': {'character': 26, 'line': 108},
           'start': {'character': 11, 'line': 108}},
 'rule': 'reportReturnType',
 'severity': 'error'}
{'message': 'Argument of type "str" cannot be assigned to parameter "action" '
            'of type "RunnableLike[Unknown, Unknown]" in function "add_node"\n'
            '\xa0\xa0Type "str" is not assignable to type '
            '"RunnableLike[Unknown, Unknown]"\n'
            '\xa0\xa0\xa0\xa0"str" is not assignable to "Runnable[Unknown, '
            'Unknown]"\